```
[Title] Multivariate LSTM
[Author] Jason Brownlee
[Etidor] Yibeck.Lee(yibeck.lee@gmail.com)
[reference]
 - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
```

In [1]:
# multivariate lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# split a multivariate sequence into samples
from numpy import array
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [3]:

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset


array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [4]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
 

In [15]:
n_features , X.shape , y.shape

(2, (7, 3, 2), (7,))

In [14]:
X

array([[[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [16]:
y

array([ 65,  85, 105, 125, 145, 165, 185])

In [5]:
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [17]:
(n_steps, n_features)

(3, 2)

In [18]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


In [19]:
# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
7/7 [==============================] - 1s 135ms/step - loss: 24272.9414
Epoch 2/200
7/7 [==============================] - 0s 0us/step - loss: 23750.4727
Epoch 3/200
7/7 [==============================] - 0s 0us/step - loss: 23123.3770
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 22428.6348
Epoch 5/200
7/7 [==============================] - 0s 0us/step - loss: 21783.6816
Epoch 6/200
7/7 [==============================] - 0s 0us/step - loss: 21128.4902
Epoch 7/200
7/7 [==============================] - 0s 0us/step - loss: 20499.4668
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 19903.3711
Epoch 9/200
7/7 [==============================] - 0s 429us/step - loss: 19361.1348
Epoch 10/200
7/7 [==============================] - 0s 571us/step - loss: 18863.0488
Epoch 11/200
7/7 [==============================] - 0s 429us/step - loss: 18413.2871
Epoch 12/200
7/7 [==============================] - 0s 429us/step - loss: 17988.3887
Epo

In [20]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=1)


1/1 [==============================] - 0s 179ms/step


In [22]:
x_input, yhat

(array([[[ 80,  85],
         [ 90,  95],
         [100, 105]]]), array([[209.72183]], dtype=float32))